<a href="https://colab.research.google.com/github/jaisidhsingh/AI-Plays-CartPole/blob/main/nn_plays_gym1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
import os
from statistics import mean, median
import random
from collections import Counter

# initialize environment 
env = gym.make('CartPole-v0')
env.reset()

array([-0.04774112, -0.03132465, -0.01949663, -0.03219317])

In [43]:
# define parameters
learning_rate = 0.1
goals = 500
score_required = 50
initial_games = 10000

# observations contains pole position, cart position etc
# rewards if the pole is balanced 1 or 0
# done if the game is over
# info is any other info

In [44]:
# play some random games as a start
def random_games():
	for x in range(5):
		env.reset()
		for i in range(goals):
			env.render()
			action = env.action_space.sample()
			observation, rewards, done, info = env.step(action)
			if done:
				break

# get training data based on random actions which get us a score of 50 or more
def initial_training():
	train_data = []
	scores = []
	accepted_scores = []

	for x in range(initial_games):
		score = 0
		memory = []
		previous_observation = []
		for i in range(goals):
			action = random.randrange(0, 2)
			observation, rewards, done, info = env.step(action)
			if len(previous_observation) > 0:
				# store previous observation with the current action
				memory.append([previous_observation, action])
			previous_observation = observation
			score += rewards
			if done:
				break
		if score >= score_required:
			accepted_scores.append(score)
			for data in memory:
				if data[1] == 1:
					output = [1 , 0] #going right
				elif data[1] == 0:
					output = [0, 1] #going left
				train_data.append([data[0], output])

		env.reset()
		scores.append(score)

	train_data_save = np.array(train_data)
	np.save('trained1.npy', train_data_save)

	print('Average accepted score : ', mean(accepted_scores), "\n")
	print('Median accepted score : ', median(accepted_scores), "\n")
	print('Max accepted score : ', max(accepted_scores), "\n")

	print(Counter(accepted_scores),"\n")

	return train_data

# prepare data to be fed into the model
def prep_train(train_data):
	X = np.array([i[0] for i in train_data])
	X = X.reshape(-1, len(train_data[0][0]))

	y = np.array([i[1] for i in train_data])
	y = y.reshape(-1, len(train_data[0][1]))

	return X, y


In [45]:
train_data = initial_training()

prepped_data = prep_train(train_data)

X_train, y_train = prepped_data

input_shape = len(X_train[0])
print(input_shape)
print(X_train.shape)
print(y_train.shape, "\n")

# reshape the data 
z = np.zeros((y_train.shape[0], 1), dtype=np.int64)
y_train = np.append(y_train, z, axis=1)
y_train = np.append(y_train, z, axis=1)

X_train = X_train.reshape((-1, 4, 1))
print(X_train.shape)
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
print(y_train.shape)

for i in range(5):
  print(X_train[i])
  print(y_train[i], "\n")

Average accepted score :  61.55334987593052 

Median accepted score :  59.0 

Max accepted score :  119.0 

Counter({50.0: 38, 53.0: 27, 51.0: 25, 55.0: 23, 59.0: 23, 54.0: 20, 52.0: 18, 57.0: 18, 56.0: 18, 63.0: 17, 60.0: 17, 62.0: 16, 64.0: 11, 68.0: 10, 65.0: 10, 58.0: 9, 61.0: 9, 66.0: 8, 67.0: 7, 69.0: 7, 72.0: 6, 71.0: 6, 73.0: 5, 70.0: 5, 75.0: 5, 74.0: 4, 87.0: 3, 80.0: 3, 99.0: 2, 94.0: 2, 86.0: 2, 85.0: 2, 76.0: 2, 82.0: 2, 89.0: 2, 79.0: 2, 119.0: 2, 88.0: 2, 81.0: 2, 78.0: 2, 83.0: 2, 91.0: 2, 77.0: 2, 111.0: 1, 84.0: 1, 115.0: 1, 90.0: 1, 101.0: 1}) 

4
(24403, 4)
(24403, 2) 

(24403, 4, 1)
(24403, 4, 1)
[[-0.04836761]
 [ 0.16407138]
 [-0.02014049]
 [-0.33096316]]
[[0]
 [1]
 [0]
 [0]] 

[[-0.04508618]
 [-0.03075817]
 [-0.02675975]
 [-0.04469901]]
[[1]
 [0]
 [0]
 [0]] 

[[-0.04570135]
 [ 0.16473708]
 [-0.02765374]
 [-0.34570332]]
[[0]
 [1]
 [0]
 [0]] 

[[-0.0424066 ]
 [-0.02998081]
 [-0.0345678 ]
 [-0.06186732]]
[[1]
 [0]
 [0]
 [0]] 

[[-0.04300622]
 [ 0.16561927]
 [-0.0358

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


'z = np.zeros((y_train.shape[0], 1), dtype=np.int64)\ny_train = np.append(y_train, z, axis=1)\ny_train = np.append(y_train, z, axis=1)\n\nX_train = X_train.reshape((-1, 4, 1))\nprint(X_train.shape)\ny_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))\nprint(y_train.shape)'

In [53]:
#define the model
def get_model():
	model = keras.models.Sequential()
	model.add(keras.layers.Dense(128, input_shape=(input_shape, 1), activation='relu'))
	model.add(keras.layers.Dropout(0.8))
	model.add(keras.layers.Dense(256, activation='relu'))
	model.add(keras.layers.BatchNormalization())
	model.add(keras.layers.Dense(512, activation='relu'))
	model.add(keras.layers.Dropout(0.8))
	model.add(keras.layers.Dense(512, activation='relu'))
	model.add(keras.layers.Dropout(0.8))
	model.add(keras.layers.Dense(256, activation='relu'))
	model.add(keras.layers.Dropout(0.8))
	model.add(keras.layers.Dense(128, activation='relu'))
	model.add(keras.layers.Dropout(0.8))
	model.add(keras.layers.BatchNormalization())
	model.add(keras.layers.Dense(1, activation='sigmoid'))

	return model

model = get_model()
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 4, 128)            256       
_________________________________________________________________
dropout_50 (Dropout)         (None, 4, 128)            0         
_________________________________________________________________
dense_74 (Dense)             (None, 4, 256)            33024     
_________________________________________________________________
batch_normalization_20 (Batc (None, 4, 256)            1024      
_________________________________________________________________
dense_75 (Dense)             (None, 4, 512)            131584    
_________________________________________________________________
dropout_51 (Dropout)         (None, 4, 512)            0         
_________________________________________________________________
dense_76 (Dense)             (None, 4, 512)          

In [49]:
# y_train = keras.utils.to_categorical(y_train, 2)
# print(y_train.shape)

In [57]:
model.compile(loss=keras.losses.binary_crossentropy,
 optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
 metrics=['accuracy'])

model_history = model.fit(X_train, y_train, batch_size=32, epochs=10)


Epoch 1/10
763/763 [==============================] - 5s 5ms/step - loss: 0.5883 - accuracy: 0.7405
Epoch 2/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5839 - accuracy: 0.7442
Epoch 3/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5795 - accuracy: 0.7464
Epoch 4/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5827 - accuracy: 0.7446
Epoch 5/10
763/763 [==============================] - 4s 6ms/step - loss: 0.5869 - accuracy: 0.7437
Epoch 6/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5845 - accuracy: 0.7430
Epoch 7/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5910 - accuracy: 0.7427
Epoch 8/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5865 - accuracy: 0.7438
Epoch 9/10
763/763 [==============================] - 4s 6ms/step - loss: 0.5837 - accuracy: 0.7437
Epoch 10/10
763/763 [==============================] - 4s 5ms/step - loss: 0.5899 - accuracy: 0.7418